<a href="https://colab.research.google.com/github/Jyo-19/FMML-2024/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
A.Increasing the percentage of the validation set:
Generally, leads to a decrease in validation accuracy.
This happens because a larger validation set means a smaller training set. With less data to learn from, the model may not generalize as well, leading to lower validation accuracy.
Reducing the percentage of the validation set:
Generally, leads to an increase in validation accuracy.
A smaller validation set means more data available for training. The model can learn more effectively from the larger training set, potentially improving its ability to generalize and resulting in higher validation accuracy.
However, keep in mind that an extremely small validation set may not accurately reflect the model's performance on unseen data.



2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
A.The size of the training and validation sets plays a crucial role in predicting test accuracy:

Training set:

Larger training set: Provides more data for the model to learn from, potentially leading to better generalization and improved accuracy on the test set.
Smaller training set: May result in the model not capturing the underlying patterns in the data, leading to lower accuracy on both the validation and test sets.
Validation set:

Larger validation set: Offers a more reliable estimate of the model's performance on unseen data, as it better represents the overall data distribution. This can lead to a more accurate prediction of test set accuracy.
Smaller validation set: Might not be representative of the overall data distribution, causing fluctuations in validation accuracy and making it less reliable for predicting test set accuracy.
The balance:

Ideally, you want a large enough training set to allow the model to learn effectively and a large enough validation set to provide a reliable estimate of test accuracy. However, you also need to consider the total amount of data available. A common split is 80% for training and 20% for validation, but this can vary depending on the specific dataset and task.



3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?
A.A good starting point is a 80/20 split (80% for training, 20% for validation). This often provides a reasonable balance between having enough data for the model to learn effectively and having a sufficiently large validation set for a reliable performance estimate.

However, the ideal percentage can depend on several factors such as:

Dataset size: For very large datasets, you might be able to use a smaller validation set (e.g., 10%) and still get a reliable estimate. For smaller datasets, you might need a larger validation set (e.g., 30%) to ensure it is representative.
Model complexity: More complex models may require more data for training, so you might need to decrease the validation set size accordingly.
Computational resources: If training is computationally expensive, a smaller validation set can reduce the overall training time.
It's often beneficial to experiment with different split percentages and see how they affect your model's performance. Techniques like cross-validation can also help you get a more robust estimate of performance with limited data.


Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


X_train = traindata
y_train = trainlabel
X_test = testdata
y_test = testlabel


knn_1 = KNeighborsClassifier(n_neighbors=1)
knn_1.fit(X_train, y_train)


y_pred_1 = knn_1.predict(X_test)
accuracy_1 = accuracy_score(y_test, y_pred_1)


knn_3 = KNeighborsClassifier(n_neighbors=3)
knn_3.fit(X_train, y_train)

y_pred_3 = knn_3.predict(X_test)
accuracy_3 = accuracy_score(y_test, y_pred_3)
print("Accuracy of 1-Nearest Neighbor Classifier: {:.2f}".format(accuracy_1))
print("Accuracy of 3-Nearest Neighbor Classifier: {:.2f}".format(accuracy_3))

Accuracy of 1-Nearest Neighbor Classifier: 0.34
Accuracy of 3-Nearest Neighbor Classifier: 0.35


## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [15]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
A.Yes, averaging validation accuracy across multiple splits generally gives more consistent and reliable results. This technique is often called cross-validation.

Here's why:

Reduces variance: A single validation split might be biased by the specific data points included or excluded. Averaging across multiple splits helps to reduce this variance and gives a more stable estimate of model performance.
Better use of data: Cross-validation allows you to use all of your data for both training and validation, unlike a single split where a portion of the data is only used for validation.
More robust evaluation: By testing your model on different data subsets, you get a more comprehensive understanding of how well it generalizes to unseen data.
Common cross-validation techniques include:

k-fold cross-validation: The data is divided into k folds, and the model is trained and validated k times, with each fold serving as the validation set once.
Stratified k-fold cross-validation: Similar to k-fold, but ensures that each fold has the same proportion of target classes as the original dataset.
Leave-one-out cross-validation: Each data point is used as a validation set once, while the rest of the data is used for training.
In summary: Averaging validation accuracy across multiple splits through cross-validation is a best practice for getting a more consistent and reliable estimate of your model's performance. It helps to mitigate the impact of data variability and gives you more confidence in your model's ability to generalize.



2. Does it give more accurate estimate of test accuracy?
A.That's a great question! While cross-validation provides a more consistent and robust estimate of model performance than a single validation split, it doesn't necessarily guarantee a more accurate estimate of the test accuracy.

Here's why:

Cross-validation accuracy is still an estimate: Even with multiple splits, you're still evaluating your model on a subset of the available data. The true test accuracy is the performance on a completely unseen dataset.
Data leakage: While less prone to bias, improper implementation of cross-validation can introduce data leakage, where information from the validation set influences the training process. This can lead to overestimation of the accuracy.
Differences between validation and test sets: The validation set used in cross-validation is still drawn from the same distribution as the training data. If the test set has a significantly different distribution, the cross-validation accuracy might not accurately reflect the true test performance.
However, cross-validation offers several advantages in estimating test accuracy:

Better generalization estimate: By evaluating on multiple folds, it gives a more comprehensive picture of how well the model generalizes, leading to a more reliable estimate compared to a single split.
Hyperparameter tuning: Cross-validation is crucial for hyperparameter tuning. By finding the best hyperparameters based on cross-validation performance, you increase the chances of achieving good test accuracy.
Model selection: Comparing the cross-validation performance of different models helps you choose the one likely to perform best on unseen data.
In summary: Cross-validation doesn't guarantee a more accurate estimate of test accuracy, but it provides a more reliable and robust estimate compared to a single validation split. It's a valuable tool for model evaluation, hyperparameter tuning, and model selection, leading to models that are more likely to generalize well to unseen data.



3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
A.You're likely referring to the number of folds (k) in k-fold cross-validation when you mention "iterations." The effect of the number of folds on the estimate is a bit of a balancing act.

Here's a breakdown of how the number of folds influences the estimate:

Higher number of folds (larger k):

Less bias: Each fold is smaller, leading to more data used for training in each iteration. This generally results in a less biased estimate of model performance as the model is trained on a larger portion of the data each time.
Higher variance: With smaller folds, the performance estimate for each fold can be more sensitive to the specific data points included, leading to higher variability in the results across folds.
More computationally expensive: More folds mean more training and evaluation cycles, increasing the computational cost.
Lower number of folds (smaller k):

Higher bias: Larger folds mean less data for training in each iteration, potentially leading to a more biased estimate.
Lower variance: With larger folds, the performance estimate for each fold is less sensitive to individual data points, resulting in lower variability across folds.
Less computationally expensive: Fewer folds reduce the computational burden.
So, do we get a better estimate with higher iterations (folds)?

Not necessarily. There's no universally optimal value for k. It depends on factors like:

Dataset size: For smaller datasets, larger k (e.g., 10-fold) might be preferable to reduce bias. For larger datasets, smaller k (e.g., 5-fold) can be sufficient and computationally more efficient.
Computational constraints: If computational resources are limited, a smaller k might be necessary.
Desired level of precision: Higher k can provide a more fine-grained estimate but might not be worth the computational cost if the difference in accuracy is negligible.
Common choices for k:

5-fold: A good starting point for many datasets, providing a reasonable balance between bias and variance.
10-fold: Often used for smaller datasets or when a more precise estimate is desired.
Leave-one-out: Suitable for very small datasets, but computationally expensive for larger ones.
In summary: The choice of k involves a trade-off between bias, variance, and computational cost. There's no one-size-fits-all answer. Experiment with different values and consider the characteristics of your dataset and available resources to determine the best k for your specific situation.


4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
A.That's an insightful question! While increasing the number of folds (iterations) in cross-validation can be helpful with small datasets, it's not a magical solution and has limitations.

Here's a more nuanced look at how increasing iterations interacts with very small datasets:

Potential Benefits:

Reduced Bias: With very small datasets, using a higher number of folds (e.g., leave-one-out cross-validation) can help reduce bias by ensuring that almost all data points are used for training in each iteration.
More Stable Estimates: While the variance might still be high due to the limited data, increasing iterations can sometimes lead to a more stable estimate of the average performance across different folds.
Limitations:

High Variance: Even with more iterations, very small datasets are inherently prone to high variance. Each fold will have very few data points, making the performance estimate for that fold highly sensitive to the specific points included.
Overfitting Risk: With limited data, increasing iterations might lead to overfitting to the validation set, especially with techniques like leave-one-out. The model might learn the idiosyncrasies of the small validation sets, leading to an overly optimistic estimate of performance.
Computational Cost: For larger datasets, the computational cost of many iterations can be prohibitive. However, with very small datasets, this might be less of a concern.
Alternatives and Considerations:

Data Augmentation: If applicable, consider data augmentation techniques to artificially increase the size of your training data.
Simpler Models: Favor simpler models with fewer parameters to reduce the risk of overfitting with limited data.
Regularization: Use regularization techniques to prevent overfitting during training.
Careful Interpretation: Be cautious when interpreting results from cross-validation with very small datasets. The estimates might still be unreliable due to the inherent limitations of the data.
In summary: Increasing iterations can be helpful to mitigate bias with very small datasets, but it's not a perfect solution. High variance and overfitting remain concerns. Consider using alternative techniques like data augmentation, simpler models, and regularization in conjunction with cross-validation. Always interpret the results with caution and awareness of the limitations imposed by the small dataset size.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, neighbours=1):

    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = KNeighborsClassifier(n_neighbors=neighbours).fit(traindata, trainlabel).predict(valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations

To observe how the accuracy of the 3 nearest neighbour classifier changes in relation to the number of splits and split size, and to compare it with the 1 nearest neighbour classifier, you'll need to conduct a series of experiments. Here's a suggested approach:

Experiment with different values:

    Number of splits: Try different values for the number of splits (e.g., 5, 10, 20).
        Split size: Vary the percentage of data used for training (e.g., 60/40, 70/30, 80/20).
        Neighbours: Compare the results for 1 nearest neighbour and 3 nearest neighbours.

    Run experiments and collect results:

For each combination of splits, split size, and neighbours, calculate the average accuracy using the AverageAccuracy function. Record the results in a table or plot them on a graph.

    Analyze the results:

    Effect of splits: How does increasing the number of splits affect the accuracy and consistency of the results for both classifiers?
        Effect of split size: How does changing the split size impact the accuracy of both classifiers?
        Comparison: How do the accuracies of the 1 nearest neighbour and 3 nearest neighbour classifiers compare across different splits and split sizes?